In [37]:
import torch
import torch.nn as nn
import torch.utils.data as Data
import torchvision

In [38]:
# torch.manual_seed(1)

In [39]:
EPOCH = 1
BATCH_SIZE = 50
LR = 0.001
DOWNLOAD_MNIST = False

In [40]:
train_data = torchvision.datasets.MNIST(root='./mnist/', train=True, transform=torchvision.transforms.ToTensor(), download=DOWNLOAD_MNIST,)
train_loader = Data.DataLoader(dataset=train_data, batch_size=BATCH_SIZE, shuffle=True)

In [41]:
test_data = torchvision.datasets.MNIST(root='./mnist/', train=False)

In [42]:
test_x = torch.unsqueeze(test_data.test_data, dim=1).type(torch.FloatTensor)[:2000].cuda()/255.
test_y = test_data.test_labels[:2000].cuda()

D:\Anaconda\lib\site-packages\torchvision\datasets\mnist.py:58: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
D:\Anaconda\lib\site-packages\torchvision\datasets\mnist.py:48: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


In [43]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Sequential(    
            nn.Conv2d(
                in_channels=1,         
                out_channels=16,      
                kernel_size=5,         
                stride=1,            
                padding=2,             
            ),                         
            nn.ReLU(),                
            nn.MaxPool2d(kernel_size=2),   
        )
        self.conv2 = nn.Sequential(         
            nn.Conv2d(16, 32, 5, 1, 2), 
            nn.ReLU(),                 
            nn.MaxPool2d(2),          
        )
        self.out = nn.Linear(32 * 7 * 7, 10)  
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = x.view(x.size(0), -1)
        output = self.out(x)
        return output

In [46]:
cnn = CNN()

In [47]:
cnn.cuda()    # Moves all model parameters and buffers to the GPU.

CNN(
  (conv1): Sequential(
    (0): Conv2d(1, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv2): Sequential(
    (0): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (out): Linear(in_features=1568, out_features=10, bias=True)
)

In [50]:
optimizer = torch.optim.Adam(cnn.parameters(), lr=LR)
loss_func = nn.CrossEntropyLoss()

In [51]:
for epoch in range(EPOCH):
    for step, (x, y) in enumerate(train_loader):
        
        b_x = x.cuda()
        b_y = y.cuda()
        
        output = cnn(b_x)            
        loss = loss_func(output, b_y) 
        optimizer.zero_grad()         
        loss.backward()               
        optimizer.step()        
        
        if step % 50 == 0:
            test_output = cnn(test_x)
            
            pred_y = torch.max(test_output, 1)[1].cuda().data # Move the computation in GPU
            
            accuracy = float((pred_y == test_y).type(torch.FloatTensor).sum()) / float(test_y.size(0))
            print('Epoch: ', epoch, '| train loss: %.4f' % loss.data.cpu().numpy(), '|test accuracy: %.2f' % accuracy)

Epoch:  0 | train loss: 2.2956 |test accuracy: 0.10
Epoch:  0 | train loss: 0.3038 |test accuracy: 0.81
Epoch:  0 | train loss: 0.4323 |test accuracy: 0.89
Epoch:  0 | train loss: 0.2538 |test accuracy: 0.91
Epoch:  0 | train loss: 0.1714 |test accuracy: 0.92
Epoch:  0 | train loss: 0.1670 |test accuracy: 0.93
Epoch:  0 | train loss: 0.2198 |test accuracy: 0.95
Epoch:  0 | train loss: 0.1572 |test accuracy: 0.95
Epoch:  0 | train loss: 0.0836 |test accuracy: 0.95
Epoch:  0 | train loss: 0.2603 |test accuracy: 0.96
Epoch:  0 | train loss: 0.0825 |test accuracy: 0.96
Epoch:  0 | train loss: 0.0660 |test accuracy: 0.96
Epoch:  0 | train loss: 0.0877 |test accuracy: 0.97
Epoch:  0 | train loss: 0.1301 |test accuracy: 0.96
Epoch:  0 | train loss: 0.0091 |test accuracy: 0.97
Epoch:  0 | train loss: 0.2104 |test accuracy: 0.96
Epoch:  0 | train loss: 0.0680 |test accuracy: 0.97
Epoch:  0 | train loss: 0.2466 |test accuracy: 0.98
Epoch:  0 | train loss: 0.1272 |test accuracy: 0.98
Epoch:  0 | 

In [52]:
test_output = cnn(test_x[:10])

In [55]:
pred_y = torch.max(test_output, 1)[1].cuda().data

print(pred_y, 'prediction number')
print(test_y[:10], 'real number')

tensor([7, 2, 1, 0, 4, 1, 4, 9, 5, 9], device='cuda:0') prediction number
tensor([7, 2, 1, 0, 4, 1, 4, 9, 5, 9], device='cuda:0') real number
